In [ ]:
# gen_txt.py
# -*- coding: utf-8 -*-
"""
文字 Prompt / 影像複製生成工具（含步驟開關 + tqdm 進度條）
支援一次處理多個類別
"""

from pathlib import Path
from PIL import Image, ImageOps
import random
import shutil
from typing import List
from tqdm import tqdm  # ✅ 進度條

# =====================【可調參數 / 路徑 / 開關】=====================

BASE_DIRS = [
    r"C:\Users\anywhere4090\Desktop\0902 finalcode\0902 finalcode\btad",
    r"C:\Users\anywhere4090\Desktop\DDAD-main\0902 finalcode\medical",
    r"C:\Users\anywhere4090\Desktop\DDAD-main\0902 finalcode\sim",
    r"C:\Users\anywhere4090\Desktop\DDAD-main\0902 finalcode\newmvtec3d",
]

TARGET_CLASSES = {
    "newmvtec3d": ["bagel"],
    "btad": [],
    "medical": [],
    "sim": [],
}

RANDOM_SEED = 42
CLIP2_COPY_NUM = 3
CLIP4_AUG_NUM  = 3
IMGSIZE = 256

IMG_EXTS = {".png", ".jpg", ".jpeg"}
TXT_SUFFIX = ".txt"
IMG_COPY_FORMAT = "{:03d}.png"
TXT_COPY_FORMAT = "{:03d}.txt"

NORMAL_PROMPTS = [
    "a flawless {}",
    "a good {}",
    "a perfect {}"
]

ANOMALY_PROMPTS = [
    "a defective {}",
    "a broken {}",
    "a damaged {}",
    "a cracked {}",
    "a faulty {}",
    "a scratched {}",
    "a {} with visible defects",
    "a malfunctioning {}",
    "a {} with flaws",
    "a deteriorated {}"
]

VERBOSE = True

RUN_GENERATE_GOOD_PROMPTS = True
RUN_GENERATE_CLIP  = False
RUN_GENERATE_CLIP2 = True
RUN_GENERATE_CLIP3 = False
RUN_GENERATE_CLIP4 = True
# ==============================================================================

class TextGenerator:
    def __init__(self, folder: str, seed: int = RANDOM_SEED,
                 clip2_aug_num: int = CLIP2_COPY_NUM,
                 clip4_aug_num: int = CLIP4_AUG_NUM):
        self.folder = Path(folder)
        if not self.folder.exists() or not self.folder.is_dir():
            raise FileNotFoundError(f"來源資料夾不存在：{self.folder}")

        random.seed(seed)
        self.images: List[Path] = sorted([p for p in self.folder.iterdir() if p.suffix.lower() in IMG_EXTS])
        if len(self.images) == 0:
            raise FileNotFoundError(f"來源資料夾找不到任何影像：{self.folder}")

        self.class_name = self.folder.parent.parent.name
        self.clip2_aug_num = int(max(1, clip2_aug_num))
        self.clip4_aug_num = int(max(1, clip4_aug_num))

        self.normal_prompts = NORMAL_PROMPTS.copy()
        self.anomaly_prompts = ANOMALY_PROMPTS.copy()

    def reset_dir(self, out_dir: Path):
        out_dir.mkdir(parents=True, exist_ok=True)
        for file in out_dir.iterdir():
            try:
                if file.is_file():
                    file.unlink()
                elif file.is_dir():
                    shutil.rmtree(file)
            except Exception as e:
                print(f"[WARN] 刪除 {file} 失敗：{e}")
        if VERBOSE:
            print(f"[INFO] 已清空資料夾：{out_dir}")

    def _save_resized(self, src_img: Image.Image, dst_img: Path):
        src_img.resize((IMGSIZE, IMGSIZE), Image.BICUBIC).save(dst_img)

    def generate_good_prompts(self):
        for img_path in tqdm(self.images, desc=f"[{self.class_name}] GOOD", unit="img"):
            txt_path = img_path.with_suffix(TXT_SUFFIX)
            prompt = random.choice(self.normal_prompts).format(self.class_name)
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(prompt)
            if VERBOSE:
                print(f"[GOOD] {img_path.name} -> {txt_path.name} : {prompt}")

    def generate_bad_prompts(self, out_dir: str):
        out_dir = Path(out_dir)
        self.reset_dir(out_dir)
        for counter, img_path in enumerate(tqdm(self.images, desc=f"[{self.class_name}] CLIP", unit="img")):
            dst_img = out_dir / IMG_COPY_FORMAT.format(counter)
            img = Image.open(img_path).convert("RGB")
            self._save_resized(img, dst_img)
            txt_path = out_dir / TXT_COPY_FORMAT.format(counter)
            prompt = random.choice(self.anomaly_prompts).format(self.class_name)
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(prompt)
            if VERBOSE:
                print(f"[BAD ] {dst_img.name} <- {img_path.name} | prompt: {prompt}")

    def augment_good_prompts(self, out_dir: str):
        out_dir = Path(out_dir)
        self.reset_dir(out_dir)
        counter = 0
        for img_path in tqdm(self.images, desc=f"[{self.class_name}] CLIP2", unit="img"):
            for k in range(self.clip2_aug_num):
                dst_img = out_dir / IMG_COPY_FORMAT.format(counter)
                img = Image.open(img_path).convert("RGB")
                self._save_resized(img, dst_img)
                txt_path = out_dir / TXT_COPY_FORMAT.format(counter)
                prompt = random.choice(self.normal_prompts).format(self.class_name)
                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(prompt)
                if VERBOSE:
                    print(f"[AUG2] {dst_img.name} <- {img_path.name} | prompt: {prompt}")
                counter += 1

    def reduce_good_prompts(self, out_dir: str):
        out_dir = Path(out_dir)
        self.reset_dir(out_dir)
        half_count = max(1, len(self.images) // 2)
        selected = random.sample(self.images, half_count)
        for idx, img_path in enumerate(tqdm(selected, desc=f"[{self.class_name}] CLIP3", unit="img")):
            dst_img = out_dir / IMG_COPY_FORMAT.format(idx)
            img = Image.open(img_path).convert("RGB")
            self._save_resized(img, dst_img)
            txt_path = out_dir / TXT_COPY_FORMAT.format(idx)
            prompt = random.choice(self.normal_prompts).format(self.class_name)
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(prompt)
            if VERBOSE:
                print(f"[HALF] {dst_img.name} <- {img_path.name} | prompt: {prompt}")

    def augment_good_clip4(self, out_dir: str):
        out_dir = Path(out_dir)
        self.reset_dir(out_dir)
        counter = 0
        for img_path in tqdm(self.images, desc=f"[{self.class_name}] CLIP4", unit="img"):
            img = Image.open(img_path).convert("RGB")
            for k in range(self.clip4_aug_num):
                angle = random.choice([0, 15, -15, 30, -30])
                scale = random.uniform(0.8, 1.2)
                w, h = img.size
                new_w, new_h = int(w * scale), int(h * scale)
                aug_img = img.resize((new_w, new_h), Image.BICUBIC).rotate(angle, expand=True)

                if random.random() < 0.5:
                    aug_img = ImageOps.mirror(aug_img)
                if random.random() < 0.3:
                    aug_img = ImageOps.flip(aug_img)

                dst_img = out_dir / IMG_COPY_FORMAT.format(counter)
                self._save_resized(aug_img, dst_img)

                txt_path = out_dir / TXT_COPY_FORMAT.format(counter)
                prompt = random.choice(self.normal_prompts).format(self.class_name)
                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(prompt)

                if VERBOSE:
                    print(f"[AUG4] {dst_img.name} <- {img_path.name} "
                          f"| angle={angle}, scale={scale:.2f} | prompt: {prompt}")
                counter += 1

# -------------------- 主程式 --------------------
if __name__ == "__main__":
    for base_dir in BASE_DIRS:
        base_name = Path(base_dir).name
        if base_name not in TARGET_CLASSES:
            continue

        for cls in TARGET_CLASSES[base_name]:
            src_dir = Path(base_dir) / cls / "train" / "good"
            if not src_dir.exists():
                print(f"[WARN] 跳過：{src_dir} 不存在")
                continue

            gen = TextGenerator(src_dir)

            if RUN_GENERATE_GOOD_PROMPTS:
                if VERBOSE: print(f"[STEP] {cls}: 產生 good 圖的 .txt")
                gen.generate_good_prompts()

            if RUN_GENERATE_CLIP:
                clip_dir = src_dir.parent / "clip"
                if VERBOSE: print(f"[STEP] {cls}: 產生 clip（bad prompts）到：{clip_dir}")
                gen.generate_bad_prompts(clip_dir)

            if RUN_GENERATE_CLIP2:
                clip2_dir = src_dir.parent / "clip2"
                if VERBOSE: print(f"[STEP] {cls}: 產生 clip2（複製 good 圖多份）到：{clip2_dir}")
                gen.augment_good_prompts(clip2_dir)

            if RUN_GENERATE_CLIP3:
                clip3_dir = src_dir.parent / "clip3"
                if VERBOSE: print(f"[STEP] {cls}: 產生 clip3（減半 good）到：{clip3_dir}")
                gen.reduce_good_prompts(clip3_dir)

            if RUN_GENERATE_CLIP4:
                clip4_dir = src_dir.parent / "clip4"
                if VERBOSE: print(f"[STEP] {cls}: 產生 clip4（旋轉+縮放+翻轉增廣 good）到：{clip4_dir}")
                gen.augment_good_clip4(clip4_dir)

    print("[DONE] 所有類別處理完成。")


[STEP] bagel: 產生 good 圖的 .txt


[bagel] GOOD: 100%|██████████| 244/244 [00:00<00:00, 5099.79img/s]


[GOOD] 000.png -> 000.txt : a perfect bagel
[GOOD] 001.png -> 001.txt : a flawless bagel
[GOOD] 002.png -> 002.txt : a flawless bagel
[GOOD] 003.png -> 003.txt : a perfect bagel
[GOOD] 004.png -> 004.txt : a good bagel
[GOOD] 005.png -> 005.txt : a flawless bagel
[GOOD] 006.png -> 006.txt : a flawless bagel
[GOOD] 007.png -> 007.txt : a flawless bagel
[GOOD] 008.png -> 008.txt : a perfect bagel
[GOOD] 009.png -> 009.txt : a flawless bagel
[GOOD] 010.png -> 010.txt : a perfect bagel
[GOOD] 011.png -> 011.txt : a perfect bagel
[GOOD] 012.png -> 012.txt : a perfect bagel
[GOOD] 013.png -> 013.txt : a flawless bagel
[GOOD] 014.png -> 014.txt : a perfect bagel
[GOOD] 015.png -> 015.txt : a good bagel
[GOOD] 016.png -> 016.txt : a flawless bagel
[GOOD] 017.png -> 017.txt : a flawless bagel
[GOOD] 018.png -> 018.txt : a flawless bagel
[GOOD] 019.png -> 019.txt : a flawless bagel
[GOOD] 020.png -> 020.txt : a flawless bagel
[GOOD] 021.png -> 021.txt : a perfect bagel
[GOOD] 022.png -> 022.txt 

[bagel] CLIP2:   1%|          | 2/244 [00:00<00:22, 10.95img/s]

[AUG2] 000.png <- 000.png | prompt: a flawless bagel
[AUG2] 001.png <- 000.png | prompt: a flawless bagel
[AUG2] 002.png <- 000.png | prompt: a perfect bagel
[AUG2] 003.png <- 001.png | prompt: a perfect bagel
[AUG2] 004.png <- 001.png | prompt: a flawless bagel
[AUG2] 005.png <- 001.png | prompt: a flawless bagel
[AUG2] 006.png <- 002.png | prompt: a flawless bagel


...

[AUG4] 713.png <- 237.png | angle=15, scale=0.80 | prompt: a perfect bagel
[AUG4] 714.png <- 238.png | angle=30, scale=0.91 | prompt: a good bagel
[AUG4] 715.png <- 238.png | angle=30, scale=0.99 | prompt: a perfect bagel
[AUG4] 716.png <- 238.png | angle=0, scale=0.87 | prompt: a flawless bagel
[AUG4] 717.png <- 239.png | angle=-30, scale=0.88 | prompt: a perfect bagel
[AUG4] 718.png <- 239.png | angle=0, scale=1.16 | prompt: a perfect bagel
[AUG4] 719.png <- 239.png | angle=-30, scale=0.96 | prompt: a perfect bagel
[AUG4] 720.png <- 240.png | angle=-15, scale=1.02 | prompt: a flawless bagel


[bagel] CLIP4:  99%|█████████▉| 242/244 [00:20<00:00, 12.11img/s]

[AUG4] 721.png <- 240.png | angle=-15, scale=0.96 | prompt: a good bagel
[AUG4] 722.png <- 240.png | angle=15, scale=0.99 | prompt: a perfect bagel
[AUG4] 723.png <- 241.png | angle=0, scale=1.07 | prompt: a perfect bagel
[AUG4] 724.png <- 241.png | angle=15, scale=1.10 | prompt: a perfect bagel
[AUG4] 725.png <- 241.png | angle=-30, scale=1.06 | prompt: a good bagel
[AUG4] 726.png <- 242.png | angle=-15, scale=0.82 | prompt: a good bagel
[AUG4] 727.png <- 242.png | angle=-15, scale=0.89 | prompt: a good bagel
[AUG4] 728.png <- 242.png | angle=30, scale=0.99 | prompt: a perfect bagel


[bagel] CLIP4: 100%|██████████| 244/244 [00:20<00:00, 12.07img/s]

[AUG4] 729.png <- 243.png | angle=-15, scale=0.84 | prompt: a good bagel
[AUG4] 730.png <- 243.png | angle=-15, scale=1.03 | prompt: a good bagel
[AUG4] 731.png <- 243.png | angle=-15, scale=1.04 | prompt: a good bagel
[DONE] 所有類別處理完成。


In [ ]:
# -*- coding: utf-8 -*-
"""
CLIP 微調訓練腳本（支援一次跑多個類別）
- 支援 good / clip / clip2 / clip3 / clip4
- clip (正常圖 + 異常文字) → 當成負樣本，會被懲罰相似度過高
- 新增 LOSS_ADAPTIVE 與 OPT_ADAPTIVE 開關
- 新增 IMGSIZE 參數，所有圖片會 resize 成同一大小
- ✅ 新增 tqdm 進度條 & CUDA debug log
"""

from pathlib import Path
from PIL import Image
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import CLIPProcessor, CLIPModel
from typing import List, Dict, Any
from tqdm import tqdm  # ✅ 進度條

# ======================【可調參數】======================
EPOCHS = 5
BATCH_SIZE = 256
LR = 5e-7
SAVE_INTERVAL = 15
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
NUM_WORKERS = 0
RESUME = False
RESUME_EPOCH = 9
CHECKPOINT_ROOT = Path("clipcheckpoints")
MODEL_PRETRAIN_NAME = "openai/clip-vit-base-patch32"
IMG_GLOB_PATTERN = "*.png"
IMGSIZE = 256
VERBOSE = True

# ===== 需要訓練的類別 =====
CATEGORIES = [
    r"C:\Users\anywhere4090\Desktop\DDAD-main\0902 finalcode\newmvtec3d\bagel",
]

# ===== 資料來源開關 =====
USE_SOURCES = {
    "good":  True,
    "clip":  False,
    "clip2": True,
    "clip3": False,
    "clip4": True,
}

# ===== 新增開關 =====
LOSS_ADAPTIVE = True
OPT_ADAPTIVE  = True

if VERBOSE:
    enabled = [k for k, v in USE_SOURCES.items() if v]
    print(f"[INFO] Enabled sources = {', '.join(enabled) if enabled else 'None'}")
    print(f"[INFO] LOSS_ADAPTIVE={LOSS_ADAPTIVE}, OPT_ADAPTIVE={OPT_ADAPTIVE}, IMGSIZE={IMGSIZE}")

# 固定隨機種子
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


# ====================== Dataset ======================
class ImageTextPairDataset(Dataset):
    def __init__(self, base_dir: str):
        self.samples: List = []
        self.class_name = Path(base_dir).name
        self.per_dir_counts = {}

        dirs = {
            "good":  (Path(base_dir) / "train/good",  USE_SOURCES["good"]),
            "clip":  (Path(base_dir) / "train/clip",  USE_SOURCES["clip"]),
            "clip2": (Path(base_dir) / "train/clip2", USE_SOURCES["clip2"]),
            "clip3": (Path(base_dir) / "train/clip3", USE_SOURCES["clip3"]),
            "clip4": (Path(base_dir) / "train/clip4", USE_SOURCES["clip4"]),
        }

        for name, (p, use_flag) in dirs.items():
            if not use_flag or not p.exists():
                continue

            if name == "clip3":
                all_imgs = sorted(p.glob(IMG_GLOB_PATTERN))
                half_count = len(all_imgs) // 2
                imgs = random.sample(all_imgs, half_count) if half_count > 0 else []
            else:
                imgs = sorted(p.glob(IMG_GLOB_PATTERN))

            count = 0
            for img_path in imgs:
                txt_path = img_path.with_suffix(".txt")
                if txt_path.exists():
                    self.samples.append((img_path, txt_path))
                    count += 1

            self.per_dir_counts[name] = count

        if VERBOSE:
            print("========== Dataset Summary ==========")
            print(f"[INFO] class_name = {self.class_name}")
            total = 0
            for name, cnt in self.per_dir_counts.items():
                print(f"[INFO] {name:5s} samples = {cnt}")
                total += cnt
            print(f"[INFO] total samples = {total}")
            print("=====================================")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, txt_path = self.samples[idx]
        img = Image.open(img_path).convert("RGB").resize((IMGSIZE, IMGSIZE))
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read().strip()
        return {"image": img, "text": text, "path": str(img_path)}


# ====================== collate / loss ======================
def collate_fn(batch, processor: CLIPProcessor) -> Dict[str, Any]:
    images = [b["image"] for b in batch]
    texts = [b["text"] for b in batch]
    paths = [b["path"] for b in batch]
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    inputs["paths"] = paths
    return inputs


def clip_loss(image_embeds, text_embeds, logit_scale, paths,
              use_clip=False, clip_neg_weight=0.5, adaptive=False):
    image_embeds = image_embeds / (image_embeds.norm(dim=-1, keepdim=True) + 1e-12)
    text_embeds  = text_embeds  / (text_embeds.norm(dim=-1, keepdim=True) + 1e-12)

    logits_per_image = torch.matmul(image_embeds, text_embeds.t()) * logit_scale
    logits_per_text  = logits_per_image.t()

    batch_size = image_embeds.size(0)
    labels = torch.arange(batch_size, device=image_embeds.device)

    loss_i = torch.nn.functional.cross_entropy(logits_per_image, labels)
    loss_t = torch.nn.functional.cross_entropy(logits_per_text, labels)
    loss = (loss_i + loss_t) / 2.0

    if use_clip:
        clip_mask = torch.tensor(
            [("train\\clip" in p.lower()) or ("train/clip" in p.lower()) for p in paths],
            dtype=torch.bool,
            device=image_embeds.device
        )
        if clip_mask.any():
            clip_img_embeds  = image_embeds[clip_mask]
            clip_text_embeds = text_embeds[clip_mask]
            if clip_img_embeds.size(0) > 0:
                neg_sim = torch.sum(clip_img_embeds * clip_text_embeds, dim=-1)
                neg_loss = torch.mean(neg_sim)
                if adaptive:
                    ratio = clip_img_embeds.size(0) / batch_size
                    loss = loss + (clip_neg_weight * ratio) * neg_loss
                else:
                    loss = loss + clip_neg_weight * neg_loss
    return loss


# ====================== Training 主流程 ======================
def train_one_category(base_dir: str):
    dataset = ImageTextPairDataset(base_dir)

    processor = CLIPProcessor.from_pretrained(MODEL_PRETRAIN_NAME)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                            collate_fn=lambda b: collate_fn(b, processor),
                            num_workers=NUM_WORKERS, pin_memory=True)

    ckpt_root = CHECKPOINT_ROOT / dataset.class_name
    ckpt_root.mkdir(parents=True, exist_ok=True)

    start_epoch = 0
    if RESUME:
        resume_path = ckpt_root / f"epoch_{RESUME_EPOCH}"
        print(f"[INFO] Resuming from {resume_path}")
        model = CLIPModel.from_pretrained(str(resume_path)).to(DEVICE)
        optimizer = AdamW(model.parameters(), lr=LR)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS) if OPT_ADAPTIVE else None
        opt_path = resume_path / "optimizer.pt"
        if opt_path.exists():
            optimizer.load_state_dict(torch.load(opt_path, map_location="cpu"))
            for state in optimizer.state.values():
                for k, v in state.items():
                    if isinstance(v, torch.Tensor):
                        state[k] = v.to(DEVICE)
        start_epoch = RESUME_EPOCH
    else:
        print(f"[INFO] Starting from scratch for class {dataset.class_name}")
        model = CLIPModel.from_pretrained(MODEL_PRETRAIN_NAME).to(DEVICE)
        optimizer = AdamW(model.parameters(), lr=LR)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS) if OPT_ADAPTIVE else None

    print(f"[DEBUG] 模型第一層 device: {next(model.parameters()).device}")
    if DEVICE.startswith("cuda"):
        print(f"[DEBUG] CUDA 可用: {torch.cuda.is_available()}, GPU 數量: {torch.cuda.device_count()}")
        print(f"[DEBUG] 當前 GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")

    model.train()
    for epoch in range(start_epoch, EPOCHS):
        total_loss = 0.0

        if DEVICE.startswith("cuda"):
            allocated = torch.cuda.memory_allocated() / 1024**2
            reserved = torch.cuda.memory_reserved() / 1024**2
            print(f"[DEBUG] Epoch {epoch+1} 啟動 - allocated={allocated:.2f}MB, reserved={reserved:.2f}MB")

        # ✅ tqdm 進度條
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch")
        for batch in pbar:
            inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "paths"}
            outputs = model(**inputs)
            loss = clip_loss(
                outputs.image_embeds,
                outputs.text_embeds,
                model.logit_scale.exp(),
                batch["paths"],
                use_clip=USE_SOURCES["clip"],
                clip_neg_weight=0.5,
                adaptive=LOSS_ADAPTIVE
            )
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += float(loss.detach().cpu().item())
            pbar.set_postfix(loss=total_loss / (pbar.n or 1))

        if scheduler:
            scheduler.step()

        avg_loss = total_loss / len(dataloader)
        print(f"[{dataset.class_name}] Epoch {epoch+1}/{EPOCHS} - Avg Loss: {avg_loss:.6f} - LR: {optimizer.param_groups[0]['lr']:.2e}")

        if (epoch + 1) % SAVE_INTERVAL == 0 or (epoch == EPOCHS - 1):
            save_path = ckpt_root / f"epoch_{epoch+1}"
            save_path.mkdir(parents=True, exist_ok=True)
            model.save_pretrained(save_path)
            processor.save_pretrained(save_path)
            torch.save(optimizer.state_dict(), save_path / "optimizer.pt")
            print(f"[{dataset.class_name}] [OK] Saved checkpoint: {save_path}")


if __name__ == "__main__":
    for cat_dir in CATEGORIES:
        train_one_category(cat_dir)

# ====================== 資料來源角色表 ======================
# good     | 正常圖 + 正常文字         | 正樣本
# clip     | 正常圖 + 壞文字           | 負樣本
# clip2    | 正常圖 + 正常文字 (副本)  | 正樣本增強
# clip3    | 正常圖 + 正常文字 (隨機半數) | 正樣本增強
# clip4    | 正常圖 + 正常文字 (另一種增強) | 正樣本增強


[INFO] Enabled sources = good, clip2, clip4
[INFO] LOSS_ADAPTIVE=True, OPT_ADAPTIVE=True, IMGSIZE=256
========== Dataset Summary ==========
[INFO] class_name = bagel
[INFO] good  samples = 244
[INFO] clip2 samples = 732
[INFO] clip4 samples = 732
[INFO] total samples = 1708
[INFO] Starting from scratch for class bagel
[DEBUG] 模型第一層 device: cuda:0
[DEBUG] CUDA 可用: True, GPU 數量: 1
[DEBUG] 當前 GPU: NVIDIA GeForce RTX 4090
[DEBUG] Epoch 1 啟動 - allocated=3097.63MB, reserved=21024.00MB


Epoch 1/5: 100%|██████████| 7/7 [00:16<00:00,  2.33s/batch, loss=6.61]


[bagel] Epoch 1/5 - Avg Loss: 5.664056 - LR: 4.52e-07
[DEBUG] Epoch 2 啟動 - allocated=4966.22MB, reserved=21024.00MB


Epoch 2/5: 100%|██████████| 7/7 [00:09<00:00,  1.33s/batch, loss=6.44]


[bagel] Epoch 2/5 - Avg Loss: 5.518414 - LR: 3.27e-07
[DEBUG] Epoch 3 啟動 - allocated=4966.22MB, reserved=21024.00MB


...

[bagel] Epoch 3/5 - Avg Loss: 5.500063 - LR: 1.73e-07
[DEBUG] Epoch 4 啟動 - allocated=4966.22MB, reserved=21024.00MB


Epoch 4/5: 100%|██████████| 7/7 [00:09<00:00,  1.33s/batch, loss=6.41]


[bagel] Epoch 4/5 - Avg Loss: 5.494154 - LR: 4.77e-08
[DEBUG] Epoch 5 啟動 - allocated=4966.22MB, reserved=21024.00MB


Epoch 5/5: 100%|██████████| 7/7 [00:09<00:00,  1.33s/batch, loss=6.41]


[bagel] Epoch 5/5 - Avg Loss: 5.492476 - LR: 0.00e+00
[bagel] [OK] Saved checkpoint: clipcheckpoints\bagel\epoch_5


In [ ]:
# -*- coding: utf-8 -*-
"""
CLIP 推論 AUROC 計算（支援一次跑多個類別，含 tqdm 進度條）
"""

from pathlib import Path
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics import roc_auc_score
import datetime
import sys
import json
from typing import Dict, Any, Optional
import re
from tqdm import tqdm  # ✅ 進度條

# =======================【可調參數區】=======================
TASKS = [
    {
        "model_dir": r"clipcheckpoints\bagel\epoch_5",
        "test_root": r"C:\Users\anywhere4090\Desktop\DDAD-main\0902 finalcode\newmvtec3d\bagel\test",
        "prompt": "a flawless bagel",
    },
]

WEIGHTS: Dict[str, float] = {"cosine": 1, "dot": 0, "l2": 0, "l1": 0}

OUT_BASE_DIR = r"C:\Users\anywhere4090\Desktop\DDAD-main"
OUT_FILE_NAME = "clip.txt"
OUT_BASE = Path(OUT_BASE_DIR)
OUT_BASE.mkdir(parents=True, exist_ok=True)
OUT_FILE = OUT_BASE / OUT_FILE_NAME

IMG_EXTS = {".png", ".jpg", ".jpeg"}
IMGSIZE = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VERBOSE = True
# ===========================================================


def _parse_model_info(model_dir: str) -> Dict[str, Optional[Any]]:
    """解析 model_dir 以取得 class_name 與 epoch"""
    p = Path(model_dir)
    info = {"class_name": None, "epoch": None, "model_dir": str(p)}
    try:
        if p.parent.name:
            info["class_name"] = p.parent.name
    except Exception:
        info["class_name"] = None
    base = p.name
    m = re.search(r"epoch[_\-]?(\d+)$", base, flags=re.IGNORECASE)
    if m:
        info["epoch"] = int(m.group(1))
        return info
    m2 = re.match(r"^(\d+)$", base)
    if m2:
        info["epoch"] = int(m2.group(1))
        return info
    for part in p.parts[::-1]:
        m3 = re.search(r"epoch[_\-]?(\d+)$", part, flags=re.IGNORECASE)
        if m3:
            info["epoch"] = int(m3.group(1))
            break
    return info


def anomaly_score(model: CLIPModel, processor: CLIPProcessor, image: Image.Image,
                  text_prompt: str, weights: Dict[str, float]) -> float:
    """計算 anomaly score"""
    if weights is None or len(weights) == 0:
        weights = {"cosine": 1.0}
    inputs = processor(images=image, text=[text_prompt], return_tensors="pt", padding=True)
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    img_emb = outputs.image_embeds
    txt_emb = outputs.text_embeds
    img_norm = img_emb / (img_emb.norm(dim=-1, keepdim=True) + 1e-12)
    txt_norm = txt_emb / (txt_emb.norm(dim=-1, keepdim=True) + 1e-12)

    scores = {}
    if "cosine" in weights:
        sim = float(torch.matmul(img_norm, txt_norm.t()).item())
        scores["cosine"] = 1.0 - sim
    if "dot" in weights:
        sim = float(torch.matmul(img_emb, txt_emb.t()).item())
        scores["dot"] = -sim
    if "l2" in weights:
        scores["l2"] = float(torch.norm(img_emb - txt_emb, p=2).item())
    if "l1" in weights:
        scores["l1"] = float(torch.norm(img_emb - txt_emb, p=1).item())

    total_w = sum(weights.values())
    if total_w == 0:
        total_w = 1.0
    total = 0.0
    for k, v in scores.items():
        w = float(weights.get(k, 0.0))
        total += v * w
    return float(total / total_w)


def run_inference_and_append_auroc(model_dir: str, test_root: str, prompt: str,
                                   weights: Dict[str, float], out_file: Path):
    """單一類別的 AUROC 推論流程（含 tqdm 進度條）"""
    model_dir = Path(model_dir)
    test_root = Path(test_root)
    if not test_root.exists():
        print(f"[ERR] test_root 不存在: {test_root}", file=sys.stderr)
        return
    try:
        processor = CLIPProcessor.from_pretrained(str(model_dir))
        model = CLIPModel.from_pretrained(str(model_dir)).to(DEVICE)
    except Exception as e:
        if VERBOSE:
            print(f"[WARN] 載入失敗 {model_dir}: {e}，改用 openai/clip-vit-base-patch32")
        processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
    model.eval()

    y_true, y_score = [], []
    num_images = 0
    per_folder_counts = {}
    subfolders = [f for f in sorted(test_root.iterdir()) if f.is_dir()]
    for folder in subfolders:
        label = 0 if folder.name.lower() == "good" else 1
        imgs = sorted([p for p in folder.iterdir() if p.suffix.lower() in IMG_EXTS])
        per_folder_counts[folder.name] = len(imgs)

        # ✅ tqdm 加入
        for img_path in tqdm(imgs, desc=f"[{folder.name}] Inference", unit="img"):
            try:
                img = Image.open(img_path).convert("RGB")
                img = img.resize((IMGSIZE, IMGSIZE), Image.BICUBIC)
                score = anomaly_score(model, processor, img, prompt, weights)
            except Exception as e:
                print(f"[ERR] {img_path} scoring failed: {e}", file=sys.stderr)
                continue
            y_true.append(label)
            y_score.append(float(score))
            num_images += 1

    if num_images == 0 or len(set(y_true)) == 1:
        auroc = float("nan")
    else:
        try:
            auroc = float(roc_auc_score(y_true, y_score))
        except Exception as e:
            print(f"[ERR] roc_auc_score failed: {e}", file=sys.stderr)
            auroc = float("nan")

    parsed = _parse_model_info(str(model_dir))
    rec = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_dir": str(model_dir),
        "parsed_class": parsed.get("class_name"),
        "parsed_epoch": parsed.get("epoch"),
        "test_root": str(test_root),
        "prompt": prompt,
        "weights": weights,
        "num_images": int(num_images),
        "per_folder_counts": per_folder_counts,
        "auroc": None if auroc is None else float(auroc),
    }

    with open(out_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    summary = f"[{rec['timestamp']}] model={rec['model_dir']} class={rec['parsed_class']} epoch={rec['parsed_epoch']} images={rec['num_images']} AUROC={rec['auroc']:.6f} prompt={rec['prompt']}"
    print(summary)


if __name__ == "__main__":
    for task in TASKS:
        run_inference_and_append_auroc(task["model_dir"], task["test_root"], task["prompt"], WEIGHTS, OUT_FILE)
    print(f"[INFO] 全部類別已完成，結果存於 {OUT_FILE}")


[hole] Inference: 100%|██████████| 21/21 [00:00<00:00, 43.31img/s]

[2025-09-20T14:12:11.743226] model=clipcheckpoints\bagel\epoch_5 class=bagel epoch=5 images=110 AUROC=0.681302 prompt=a flawless bagel
[INFO] 全部類別已完成，結果存於 C:\Users\anywhere4090\Desktop\DDAD-main\clip.txt
